In [12]:
import sys, os, glob 
import datetime, time

import numpy as np
from numpy import linalg as nla
import matplotlib.pyplot as plt


gen_fn_dir = os.path.abspath('..') + '/2020_08_nonsymm_functions/'
sys.path.append(gen_fn_dir)
import nonsymm_graph_fns as ngf
import nonsymm_sparsification_fns as nsf
import linear_net_fns as lnf

gen_fn_dir = os.path.abspath('..') + '/2020_02_general_functions/'
sys.path.append(gen_fn_dir)
import lin_alg_fns as laf
import figure_fns as ff
import undirected_graph_fns as gf
import sparsification_fns as sf
from general_file_fns import load_pickle_file, save_pickle_file

%matplotlib inline

curr_date=datetime.datetime.now().strftime('%Y_%m_%d')+'_'

curr_seed=int((time.time()%1)*(2**31))
np.random.seed(curr_seed)

In [13]:
#parameters
n_nodes = 10000
clusters = [6600, 1000, 1600, 100, 100, 100, 100, 100, 100, 100, 100] #cluster sizes
sr_density = 0.6 #density of connections within clusters
lr_tot = 5000 #total num of connections across clusters
B_no_diag = gf.make_imb_clustered_graph(clusters, sr_density, lr_tot) #makes connectivity matrix, to be shifted
desired_edge_frac = 0.03

In [ ]:
# Make a connectivity matrix to use
t0 = time.time()
#diag_pad = np.diag(0.05 * np.ones(n_nodes))
A = laf.shift_to_make_diagonally_dominant(B_no_diag, shift_type='individual', diag_type='neg')# + diag_pad #CAN EDIT THIS IF WORRIED ABOUT DIAG, currently takes abs row sum as diag (note asymmetry)
ew_orig, ev_orig = laf.sort_eig(A, ret_ev=True)
print("old smallest eigvals are {}".format(np.real(ew_orig[-10:])))

# Check that S is not too close to singular, both for the sparsification and for the simulation dynamics
# (which would be super slow if S was near singular). Note that if the entries of S were all positive then
# we'd be back in the graph Laplacian-like case and S would be quite close to singular.
if np.min(np.abs(ew_orig))<1e-10:
    print('Matrix close to singular be careful')
t1 = time.time()

In [ ]:
graph_name = '{}n{}_directed_clustered_{}_sr_d{}_lr_t{}_b-{}_ics-{}'.format(curr_date,n_nodes,clusters,
                                                                              sr_density,lr_tot,  'constant', 'random')

#make the directory for this graph, uncomment if directory does not exist yet!
os.makedirs('{}'.format(graph_name))

In [ ]:
# Set up the sparsification
n_edges_orig = (np.count_nonzero(A) - n_nodes)/2 #got rid of division by 2 since each direction is an edge now
desired_mean_num_edges = desired_edge_frac * n_edges_orig # Leave as a float for now since used in normalization
print(desired_mean_num_edges)
prob_normalization = {'norm_type' : 'sum', 'norm_val' : desired_mean_num_edges, 'max_val' : 1}
prob_matrix, inv_diff_matrix = sf.get_sparsify_probs(A, ret_type=('samp_probs_and_diffs', 'matrix'), normalization=prob_normalization)
control_prob_matrix = sf.get_sparsify_probs_control(A, normalization=prob_normalization)
# _md indicates matched diagonal (i.e., comes from sampling) #only going to worry about md case for now
A_sparse_md = sf.sparsify_given_probs(A, prob_matrix, diagonal_type='row_sum')
# _wbc indicates weight based control
A_sparse_wbc = sf.sparsify_given_probs(A, control_prob_matrix, diagonal_type='row_sum') #weight_based control
diag_idx = np.diag_indices(n_nodes)
n_edges_sparse = (np.count_nonzero(A_sparse_md) - n_nodes)/2
print('Number of edges original {}, new {}, ratio {}'.format(n_edges_orig, n_edges_sparse, 
                                                             n_edges_sparse/n_edges_orig))

In [ ]:
# Calculate some measures for how much the spectrum has changed
spect_md = nsf.compare_spectrum(A_sparse_md, ew_orig, ev_orig, ret_ew=True)
spect_wbc = nsf.compare_spectrum(A_sparse_wbc, ew_orig, ev_orig, ret_ew=True)
print(spect_md.keys())

In [ ]:
ew_bds = [np.min(ew_orig), np.max(ew_orig)]
fig, ax = plt.subplots(1,2,figsize=(12,6))
ax[0].scatter(ew_orig, spect_md['ew'])
ax[0].plot(ew_bds, ew_bds, color='k')
ax[1].scatter(ew_orig, spect_wbc['ew'])
ax[1].plot(ew_bds, ew_bds, color='k')

im_ct=0
#fig.savefig('{}/{}_ew_preservation.pdf'.format(graph_name,im_ct))
im_ct+=1

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(16,6))
ax[0].boxplot([np.abs(spect_md['eps_ew']), np.abs(spect_wbc['eps_ew'])], labels = ['noise-prune', 'weights']);
ax[0].set_ylabel('$\epsilon_{\lambda_i}$', fontsize = 14);
ax[1].boxplot([np.abs(spect_md['eps_ev']), np.abs(spect_wbc['eps_ev'])], labels = ['noise-prune', 'weights']);
ax[1].set_ylabel('$\epsilon_{v_i}$', fontsize = 14);
ax[2].boxplot([np.abs(spect_md['S_ev_angle']), np.abs(spect_wbc['S_ev_angle'])], labels = ['noise-prune', 'weights']);
ax[2].set_ylabel('$\cos{\Theta_i}$', fontsize = 14);


fig.savefig('{}/{}_boxplots.pdf'.format(graph_name, im_ct))
im_ct+=1

#ff.ref_axes(ax[0], labels=False)
#ff.ref_axes(ax[1], labels=False)
#ff.ref_axes(ax[2], labels=False)
#fig.savefig('{}/{}_boxplots.pdf'.format(graph_name, im_ct))

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(16,6))
ax[0].violinplot([np.abs(spect_md['eps_ew']), np.abs(spect_wbc['eps_ew'])], bw_method = 4)#, labels = ['noise-prune', 'weights']);
ax[0].set_ylabel('$\epsilon_{\lambda_i}$', fontsize = 14);
ax[1].violinplot([np.abs(spect_md['eps_ev']), np.abs(spect_wbc['eps_ev'])], bw_method = 4)#, labels = ['noise-prune', 'weights']);
ax[1].set_ylabel('$\epsilon_{v_i}$', fontsize = 14);
ax[2].violinplot([np.abs(spect_md['S_ev_angle']), np.abs(spect_wbc['S_ev_angle'])], bw_method = 4)#, labels = ['noise-prune', 'weights']);
ax[2].set_ylabel('$\cos{\Theta_i}$', fontsize = 14);


fig.savefig('{}/{}_boxplots.pdf'.format(graph_name, im_ct))
im_ct+=1

#ff.ref_axes(ax[0], labels=False)
#ff.ref_axes(ax[1], labels=False)
#ff.ref_axes(ax[2], labels=False)
#fig.savefig('{}/{}_boxplots.pdf'.format(graph_name, im_ct))

In [ ]:
flierprops = dict(marker='o', markersize=3.5,
                  linestyle='none')

fig, ax = plt.subplots(1,3,figsize=(8,3))
fig.subplots_adjust(wspace=0.45)
ax[0].violinplot([np.abs(spect_md['eps_ew']), np.abs(spect_wbc['eps_ew'])], bw_method = 4)#, labels = ['noise-prune', 'weights']);
ax[0].set_ylabel('$\epsilon_{\lambda_i}$', fontsize = 14);
ax[1].violinplot([np.abs(spect_md['eps_ev']), np.abs(spect_wbc['eps_ev'])], bw_method = 4)#, labels = ['noise-prune', 'weights']);
ax[1].set_ylabel('$\epsilon_{v_i}$', fontsize = 14);
ax[2].violinplot([np.abs(spect_md['S_ev_angle']), np.abs(spect_wbc['S_ev_angle'])], bw_method = 4)#, labels = ['noise-prune', 'weights']);
ax[2].set_ylabel('$\cos{\Theta_i}$', fontsize = 14);
ax[0].boxplot([np.abs(spect_md['eps_ew']), np.abs(spect_wbc['eps_ew'])], flierprops=flierprops, labels = ['noise-prune', 'weights']);
ax[0].set_ylabel('$\epsilon_{\lambda_i}$', fontsize = 14);
ax[1].boxplot([np.abs(spect_md['eps_ev']), np.abs(spect_wbc['eps_ev'])], flierprops=flierprops, labels = ['noise-prune', 'weights']);
ax[1].set_ylabel('$\epsilon_{v_i}$', fontsize = 14);
ax[2].boxplot([np.abs(spect_md['S_ev_angle']), np.abs(spect_wbc['S_ev_angle'])], flierprops=flierprops, labels = ['noise-prune', 'weights']);
ax[2].set_ylabel('$\cos{\Theta_i}$', fontsize = 14);


fig.savefig('{}/{}_boxplots.pdf'.format(graph_name, im_ct))
im_ct+=1

#ff.ref_axes(ax[0], labels=False)
#ff.ref_axes(ax[1], labels=False)
#ff.ref_axes(ax[2], labels=False)
#fig.savefig('{}/{}_boxplots.pdf'.format(graph_name, im_ct))

In [ ]:
print(np.mean(np.abs(spect_md['S_ev_angle'])))
print(np.mean(np.abs(spect_wbc['S_ev_angle'])))

In [ ]:
print(len(spect_md['S_ev_angle']))
print(len(spect_wbc['S_ev_angle']))

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(15,10))
ax[0,0].hist(spect_md['eps_ew'],bins = 'auto');
ax[0,1].hist(spect_md['eps_ev'],bins = 'auto');
ax[0,2].hist(spect_md['S_ev_angle'],bins = 'auto');
ax[1,0].hist(spect_wbc['eps_ew'],bins = 'auto');
ax[1,1].hist(spect_wbc['eps_ev'],bins = 'auto');
ax[1,2].hist(spect_wbc['S_ev_angle'],bins = 'auto');

In [ ]:
#plot some of the matrices themselves
fig, axs = plt.subplots(2, 2,figsize=(15,10))
cm = 'viridis'
#cm = 'RdBu_r'
D = inv_diff_matrix# + np.transpose(inv_diff_matrix)
Matrices = [[A - np.diag(np.diag(A)),
            D], 
            [prob_matrix,
            control_prob_matrix]]

Labels = [['$|A|$',
            '$D$'],
            ['$P_{noise}$',
            '$P_{control}$']]

for col in range(2):
    for row in range(2):
        ax = axs[row, col]
        ax.set_title(Labels[row][col])
        pcm = ax.matshow(Matrices[row][col],
                            cmap=cm)
        fig.colorbar(pcm, ax=ax)

path = os.getcwd()
print(path)

#fig.savefig('{}/{}_matrix_visual.pdf'.format(graph_name, im_ct))
im_ct+=1